## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: GM12878
# EMBRACENET

da fare 
- sistema selezione augmentation in embracenet
- crea grafici AUPRC reti

In [1]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [3]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [4]:
cell_line = CELL_LINES[1]
cell_line

'GM12878'

---

In [5]:
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline
from BIOINF_tesi.models import EmbraceNetMultimodal, ConcatNetMultimodal
from BIOINF_tesi.models.utils import Kfold_CV_Multimodal

In [6]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [48]:
task = TASKS[0]

In [7]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [8]:
model=EmbraceNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


[I 2021-11-05 08:43:08,914] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_1' instead of creating a new one.
/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.434053902049873
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.0
    FFNN_n_layers: 4
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 4
    lr: 7.66660678889529e-05
    n_post_layers: 1
    optimizer: Nadam
    selection_probabilities_FFNN: 0.9924319927827241
    weight_decay: 0.0006775329255589683

===============> MODEL TESTING
AUPRC test score: 0.44048713128901873




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:11,107] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.44127670363780414
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 512
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.0
    FFNN_n_layers: 4
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 64
    FFNN_n_units_l2: 4
    FFNN_n_units_l3: 16
    lr: 0.0005364340026237666
    n_post_layers: 1
    optimizer: Nadam
    selection_probabilities_FFNN: 0.7827201154802402
    weight_decay: 0.00019872184172093634

===============> MODEL TESTING
AUPRC test score: 0.4026398868594192


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:11,454] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4102320351740525
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 512
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.05826595186832341
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.6760901969427707
    weight_decay: 0.0023383660848408246

===============> MODEL TESTING
AUPRC test score: 0.18708350165182097



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.3434


In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [11]:
model=ConcatNetMultimodal

In [14]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:11,945] Using an existing study with name 'GM12878_active_E_vs_inactive_E_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.42024697614599094
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 256
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.5
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 512
    CONCATNET_n_units_l2: 64
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 7.429220601630705e-05
    optimizer: RMSprop
    weight_decay: 0.0006554165543963894

===============> MODEL TESTING
AUPRC test score: 0.3331173078456278




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:12,506] Using an existing study with name 'GM12878_active_E_vs_inactive_E_ConcatNetMultimodal_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.30535362242399067
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.2
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 64
    CONCATNET_n_units_l2: 128
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 1.6059836015992958e-05
    optimizer: Nadam
    weight_decay: 0.002211348516713555

===============> MODEL TESTING
AUPRC test score: 0.4207583551164777




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:13,238] Using an existing study with name 'GM12878_active_E_vs_inactive_E_ConcatNetMultimodal_3' instead of creating a new one.


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.22591169158398247
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.5
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 256
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 16
    lr: 0.0013116688555601344
    optimizer: Adam
    weight_decay: 0.0011201081017597947

===============> MODEL TESTING
AUPRC test score: 0.373335672481483



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.37574


In [15]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [16]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [35]:
model=EmbraceNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (46002) in class 0 will be larger than the number of samples in the majority class (class #0 -> 23001)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:45:32,620] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySumma

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.19123624809098355
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 5.857045627898654e-05
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.7574436831231909
    weight_decay: 0.039205338115705285

===============> MODEL TESTING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (68936) in class 0 will be larger than the number of samples in the majority class (class #0 -> 34468)
  warnings.warn(


AUPRC test score: 0.1911440028660529


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


...

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.18132042253521127
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 128
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.0021363699135981567
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.34368766398643236
    weight_decay: 0.025875739743203625

===============> MODEL TESTING
AUPRC test score: 0.18309334657398224


>>> ITERATION N. 3

============

/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (45992) in class 0 will be larger than the number of samples in the majority class (class #0 -> 22996)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:47:15,408] Using an existing study with name 'GM12878_active_E_vs_inactive_E_EmbraceNetMultimodal_3augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.43529699516611925
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 256
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.0
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 32
    EMBRACENET_n_units_l1: 128
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 32
    lr: 0.0005538051383889666
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.7317029636086937
    weight_decay: 0.0011618132961793042

===============> MODEL TESTING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (68916) in class 0 will be larger than the number of samples in the majority class (class #0 -> 34458)
  warnings.warn(


AUPRC test score: 0.43224564553345624



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.26883


In [11]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [11]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [9]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [7]:
model=EmbraceNetMultimodal

In [20]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:14,431] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.34078187636823737
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 256
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 16
    lr: 0.00112518822935567
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.639723824863646
    weight_decay: 0.00019049207279029588

===============> MODEL TESTING
AUPRC test score: 0.3444556834017955




/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:14,945] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.24301440572147803
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 64
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    lr: 0.0007525113112345566
    n_post_layers: 1
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.595163316596057
    weight_decay: 0.00043071436478171217

===============> MODEL TESTING
AUPRC test score: 0.3565717869507904


>>> ITERATION N. 3


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:15,395] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3276245418145691
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.009596327515413283
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.6636642042998211
    weight_decay: 0.004738254358577602

===============> MODEL TESTING
AUPRC test score: 0.33895334462060034



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.34666


In [21]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [22]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [12]:
model=ConcatNetMultimodal

In [24]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:15,907] Using an existing study with name 'GM12878_active_P_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.31483906221944097
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CONCATNET_dropout_l0: 0.3
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 512
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 64
    lr: 3.3034310994377605e-05
    optimizer: RMSprop
    weight_decay: 0.0004399620883988981

===============> MODEL TESTING
AUPRC test score: 0.31651919343072993


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:16,329] Using an existing study with name 'GM12878_active_P_vs_inactive_P_ConcatNetMultimodal_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3521991718097556
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_dropout_l1: 0.5
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 32
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 0.0019392449797242788
    optimizer: RMSprop
    weight_decay: 0.0017437142929180862

===============> MODEL TESTING
AUPRC test score: 0.3562384715170718


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:16,737] Using an existing study with name 'GM12878_active_P_vs_inactive_P_ConcatNetMultimodal_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2089317079993845
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.5
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 128
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.5
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 16
    lr: 0.01440272176617833
    optimizer: Nadam
    weight_decay: 0.00521247553751171

===============> MODEL TESTING
AUPRC test score: 0.2162942136016667



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.29635


In [25]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [26]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [39]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (76218) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38109)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:48:29,886] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_1augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (114610) in class 0 will be larger than the number of samples in the majority class (

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.21253562446608262
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 32
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 64
    lr: 0.0006529636473777145
    n_post_layers: 1
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.5473965392569035
    weight_decay: 0.001396625595818958

===============> MODEL TESTING
AUPRC test score: 0.23659161867678605


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (76406) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38203)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:50:23,677] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_2augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3501731145924581
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.3
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    lr: 0.0007122641707093011
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.9812496642567293
    weight_decay: 0.0002782035416725741

===============> MODEL TESTING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (114388) in class 0 will be larger than the number of samples in the majority class (class #0 -> 57194)
  warnings.warn(


AUPRC test score: 0.3386974148486699


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (76238) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38119)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:52:17,837] Using an existing study with name 'GM12878_active_P_vs_inactive_P_EmbraceNetMultimodal_3augmentation' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (114382) in class 0 will be larger than the number of samples in the majority class (

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.22047835820430478
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 256
    CNN_out_channels_l3: 512
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    lr: 3.3221917099914835e-05
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.3903745850920022
    weight_decay: 0.011166954802817972

===============> MODEL TESTING
AUPRC test score: 0.2355

In [17]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [18]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [11]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [19]:
model=EmbraceNetMultimodal

In [30]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:17,509] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.8670585993072386
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 96
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 64
    lr: 0.00018507251867082717
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.8780774090675403
    weight_decay: 0.00027381383420086017

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:17,746] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_2' instead of creating a new one.


AUPRC test score: 0.8547835355586682


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:17,969] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_3' instead of creating a new one.


  Value:  0.8297084151579075
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 5
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 512
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.007720376950978735
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.5729999874889647
    weight_decay: 0.0021661727893409165

===============> MODEL TESTING
AUPRC test score: 0.8263518368037645


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5686504822131878
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_dro

In [31]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [32]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [22]:
model=ConcatNetMultimodal

In [34]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:18,307] Using an existing study with name 'GM12878_active_E_vs_active_P_ConcatNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.8258897357511789
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 64
    CNN_out_channels_l3: 256
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.0
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 32
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 64
    lr: 0.0001880696182080898
    optimizer: Nadam
    weight_decay: 0.001560888043383557

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:18,572] Using an existing study with name 'GM12878_active_E_vs_active_P_ConcatNetMultimodal_2' instead of creating a new one.


AUPRC test score: 0.6448592534704417


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.8308164866950468
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    CONCATNET_dropout_l0: 0.3
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.3
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 128
    CONCATNET_n_units_l2: 32
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 8.555657716671849e-05
    optimizer: Adam
    weight_decay: 0.0025792878032937206

===============> MODEL TESTING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:18,811] Using an existing study with name 'GM12878_active_E_vs_active_P_ConcatNetMultimodal_3' instead of creating a new one.


AUPRC test score: 0.8220648483644434


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.8301165651408803
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.5
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 256
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.0023685542965349044
    optimizer: Nadam
    weight_decay: 0.0012146647697097407

===============> MODEL TESTING
AUPRC test score: 0.8277086698904819



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.76488


In [35]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [36]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (12566) in class 0 will be larger than the number of samples in the majority class (class #0 -> 6283)
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (10210) in class 1 will be larger than the number of samples in the majority class (class #0 -> 6283)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:53:31,416] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_1augmentation' 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7896004583138194
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 128
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 3.4124962310534034e-05
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.07165273287630936
    weight_decay: 0.000753062533980122

===============> MODEL TESTING
AUPRC test score: 0.4658144571571193


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (12516) in class 0 will be larger than the number of samples in the majority class (class #0 -> 6258)
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (10260) in class 1 will be larger than the number of samples in the majority class (class #0 -> 6258)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:53:36,836] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_2augmentation' 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7882274997877516
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 512
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 16
    lr: 4.713150907170349e-05
    n_post_layers: 1
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.09611793384065914
    weight_decay: 0.04338349413384123

===============> MODEL TESTING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (18682) in class 0 will be larger than the number of samples in the majority class (class #0 -> 9341)
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (15484) in class 1 will be larger than the number of samples in the majority class (class #0 -> 9341)
  warnings.warn(


AUPRC test score: 0.7640889477500337


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (12532) in class 0 will be larger than the number of samples in the majority class (class #0 -> 6266)
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (10246) in class 1 will be larger than the number of samples in the majority class (class #0 -> 6266)
  warnings.warn(
/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-01 08:53:42,972] Using an existing study with name 'GM12878_active_E_vs_active_P_EmbraceNetMultimodal_3augmentation' 

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.8433926658786626
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 128
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 64
    lr: 0.019418658001107626
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.3909945705111536
    weight_decay: 0.00010931307392824462

===============> MODEL TESTING
AUPRC test score: 0.7951692521224517



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.67502


In [23]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [15]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [13]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [26]:
model=EmbraceNetMultimodal

In [40]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:20,906] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5449487745914114
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_dropout_l1: 0.0
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 3.646388320796877e-05
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.5243890254915275
    weight_decay: 0.06229862569412388

===============> MODEL TESTING
AUPRC test score: 0.5392898002473241


>>> ITERATION N. 2


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:21,702] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_EmbraceNetMultimodal_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5633318536073797
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 4
    lr: 1.1694228749755962e-05
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.24152268715197422
    weight_decay: 0.0012043661982234029

===============> MODEL TESTING
AUPRC test score: 0.5420239153105089


>>> ITERATION N. 3


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:22,455] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_EmbraceNetMultimodal_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5826634424976603
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 128
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.000664416243243537
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.358981634398834
    weight_decay: 0.00010921870994511394

===============> MODEL TESTING
AUPRC test score: 0.5979967409139233



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.55977


In [41]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [42]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [16]:
model=ConcatNetMultimodal

In [10]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 08:09:31,315] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_ConcatNetMultimodal_1' instead of creating a new one.
/opt/conda/lib/python3.7/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5424169979295155
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.3
    CONCATNET_dropout_l2: 0.5
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 512
    CONCATNET_n_units_l1: 512
    CONCATNET_n_units_l2: 16
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.4
    FFNN_dropout_l3: 0.5
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 16
    lr: 0.0002601352269233871
    optimizer: Adam
    weight_decay: 0.024930374625176378

===============> MODEL TESTING
AUPRC test score: 0.5433751994298683


>>> ITERATION N. 2


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 08:09:33,991] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4233390496136193
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.5
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    CNN_out_channels_l3: 512
    CONCATNET_dropout_l0: 0.5
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 1024
    FFNN_dropout_l0: 0.3
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.01410960588607877
    optimizer: Nadam
    weight_decay: 0.018712221008761333

===============> MODEL TESTING
AUPRC test score: 0.4951939320700933


>>> ITERATION N. 3


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-28 08:09:34,663] Using an existing study with name 'GM12878_inactive_E_vs_inactive_P_ConcatNetMultimodal_1_2_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5700257059475923
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 32
    CONCATNET_dropout_l0: 0.2
    CONCATNET_dropout_l1: 0.2
    CONCATNET_dropout_l2: 0.5
    CONCATNET_n_post_layers: 3
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 64
    CONCATNET_n_units_l2: 256
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.0015129050874388707
    optimizer: Adam
    weight_decay: 0.0003263355135051064

===============> MODEL TESTING
AUPRC test score: 0.6041115455661381



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.54756


In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [43]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (76346) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38173)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (45912) in class 1 will be larger than the number of samples in the majority class (class #0 -> 38173)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-31 09:03:23,436] Using an exist

Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5707890223123847
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.2
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 128
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 4
    lr: 6.307192144237503e-05
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.3759214436161208
    weight_decay: 0.000307910250244498

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (114520) in class 0 will be larger than the number of samples in the majority class (class #0 -> 57260)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (68868) in class 1 will be larger than the number of samples in the majority class (class #0 -> 57260)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.5747756137519461


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


...

AUPRC test score: 0.5822503358664414


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (76386) in class 0 will be larger than the number of samples in the majority class (class #0 -> 38193)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (45872) in class 1 will be larger than the number of samples in the majority class (class #0 -> 38193)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-31 09:08:13,049] Using an exist

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5293026402310934
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 256
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 128
    EMBRACENET_n_units_l1: 64
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 16
    lr: 0.0006045531832461281
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.3088811573737791
    weight_decay: 0.020936922676757334

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (114576) in class 0 will be larger than the number of samples in the majority class (class #0 -> 57288)
  f"After over-sampling, the number of samples ({n_samples})"
/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (68812) in class 1 will be larger than the number of samples in the majority class (class #0 -> 57288)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.5338093661098643



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.56361


In [17]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [7]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [8]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [30]:
model=EmbraceNetMultimodal

In [46]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:25,115] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.22322227905653333
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 512
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.04059614929791728
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.1371380113979449
    weight_decay: 0.0008374159867335739

===============> MODEL TESTING
AUPRC test score: 0.16012271953754026


>>> ITERATION N. 2


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:25,908] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.32437125529648786
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 512
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.0
    FFNN_dropout_l3: 0.5
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 64
    FFNN_n_units_l3: 16
    lr: 1.5709965994881367e-05
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.4746349240743316
    weight_decay: 0.00014528235718391393

===============> MODEL TESTING
AUPRC test score: 0.33246522069211004


>>> ITERATION N. 3


/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-05 08:43:26,568] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2237428499640382
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 96
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.2
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 4
    lr: 0.001296037415539155
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.0007827914268795366
    weight_decay: 0.0043564826671396726

===============> MODEL TESTING
AUPRC test score: 0.19660325704649204



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.22973


In [47]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [48]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [9]:
model=ConcatNetMultimodal

In [58]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-06 09:47:25,632] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_ConcatNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.32132369446785664
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CONCATNET_dropout_l0: 0.0
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 768
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 32
    lr: 0.0011065841111600257
    optimizer: Nadam
    weight_decay: 0.0015236772943689346

===============> MODEL TESTING
AUPRC test score: 0.344616236272088


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-06 09:47:25,934] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_ConcatNetMultimodal_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3138278531189664
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 15
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 256
    CONCATNET_dropout_l0: 0.0
    CONCATNET_n_post_layers: 1
    CONCATNET_n_units_l0: 768
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 0.0018730914294376093
    optimizer: Nadam
    weight_decay: 0.028118080021789567

===============> MODEL TESTING
AUPRC test score: 0.3213145880724536


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-11-06 09:47:26,192] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_ConcatNetMultimodal_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.20662936748324492
  Params: 
    CNN_dropout_l0: 0.3
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    CONCATNET_dropout_l0: 0.5
    CONCATNET_dropout_l1: 0.3
    CONCATNET_n_post_layers: 2
    CONCATNET_n_units_l0: 768
    CONCATNET_n_units_l1: 64
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.4
    FFNN_n_layers: 4
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 64
    FFNN_n_units_l3: 32
    lr: 0.0038779015991115278
    optimizer: RMSprop
    weight_decay: 0.007759199095934482

===============> MODEL TESTING
AUPRC test score: 0.18483445121651373



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.28359


In [59]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [60]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
## AUGMENTATION

### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=True,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}_augmentation',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_augmentation_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (122456) in class 0 will be larger than the number of samples in the majority class (class #0 -> 61228)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-31 09:11:08,211] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3227338675636787
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.3
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 4
    lr: 0.0009241627898406429
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.8674282225472876
    weight_decay: 0.0009144381423170731

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (183720) in class 0 will be larger than the number of samples in the majority class (class #0 -> 91860)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.3378884985894605


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


...

AUPRC test score: 0.33539746115477914


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (122220) in class 0 will be larger than the number of samples in the majority class (class #0 -> 61110)
  f"After over-sampling, the number of samples ({n_samples})"
/home/jupyter/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-31 09:19:49,707] Using an existing study with name 'GM12878_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_augmentation_2_augmentation_3_augmentation' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.28257859765236387
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 16
    lr: 0.00035684890050965886
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.165237756745383
    weight_decay: 0.0005989069955678149

===============> MODEL TESTING


/opt/conda/lib/python3.7/site-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (183118) in class 0 will be larger than the number of samples in the majority class (class #0 -> 91559)
  f"After over-sampling, the number of samples ({n_samples})"


AUPRC test score: 0.2821148193083256



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.31847


In [23]:
results_dict[cell_line][task][f'{model.__name__}_augm'] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---
---

## FREEZED LAYERS

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

## NOT FREEZED LAYERS + AUGMENTATION

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

In [ ]:
plot_scores(cells=cell_line)